In [1]:
from qiskit import *
from qiskit.tools.monitor import job_monitor
from qiskit.visualization.pulse_v2 import draw
# from qiskit.visualization import plot_histogram
import numpy as np
from qiskit import transpile, schedule as build_schedule

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-skku', group='snu', project='snu-students')

backend = provider.get_backend('ibm_lagos')

In [2]:
def Circuit_to_schedule(circ, draw = False):
    schedule = build_schedule(circ, backend)
    if draw:
        schedule.draw()
    return schedule

In [3]:
#dynamic decoupling 없는 ansatz circuit
def Circ_no_decoupling(circuit, t, initial):
    circuit = QuantumCircuit(5,2)
    
    circuit.barrier()
    circuit.cx(0,1)

    circuit.sx(0)
    circuit.rz(-np.pi/2,0)
    circuit.sx(0)
    circuit.x(0)

    circuit.cx(1,0)
    circuit.rz(-t/2,0)
    circuit.cx(1,0)

    circuit.rz(t/2,0)
    circuit.sx(0)
    circuit.rz(np.pi/2,0)
    circuit.sx(0)
    circuit.x(0)
    circuit.cx(0,1)
    
    circuit.barrier()
    
    return circuit

# c = transpile(Circ_no_decoupling(np.pi), backend = backend, optimization_level = 0, initial_layout = [1,3,0,2,5])
# Circuit_to_schedule(c).draw()

In [4]:
#dynamic decoupling 있는 ansatz circuit
def Circ_decoupling(t):
    circuit = QuantumCircuit(5,2)
    
    #first CNOT
    circuit.barrier()
    circuit.x(2)
    circuit.x(3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.delay(160,2)
    circuit.delay(160,3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.delay(512,2)
    circuit.delay(512,3)
    circuit.cx(0,1)
    circuit.barrier()

    circuit.sx(0)
    circuit.rz(-np.pi/2,0)
    circuit.sx(0)
    circuit.x(0)
    circuit.barrier()
    
    #second CNOT
    circuit.x(4)
    circuit.rz(np.pi,4)
    circuit.x(4)

    circuit.delay(160,4)
    circuit.rz(np.pi,4)
    circuit.x(4)
    circuit.cx(1,0)
    circuit.delay(352,4)
    circuit.barrier()

    circuit.rz(-t/2,0)

    circuit.barrier()

    #third CNOT
    circuit.x(4)
    circuit.rz(np.pi,4)
    circuit.x(4)

    circuit.delay(160,4)

    circuit.x(4)

    circuit.rz(np.pi,4)
    
    circuit.cx(1,0)
    circuit.delay(352,4)
    circuit.barrier()

    circuit.rz(t/2,0)
    circuit.sx(0)
    circuit.rz(np.pi/2,0)
    circuit.sx(0)
    circuit.x(0)
    circuit.barrier()

    #fourth CNOT
    circuit.x(2)
    circuit.x(3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.delay(160,2)
    circuit.delay(160,3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.cx(0,1)
    circuit.delay(512,2)
    circuit.delay(512,3)
    circuit.barrier()

    return circuit

# c = transpile(Circ_decoupling(np.pi), backend = backend, optimization_level = 0, initial_layout = [1,3,0,2,5])
# Circuit_to_schedule(c).draw()

In [6]:
#dynamic decoupling 있는 CNOT circuit
def CNOT_circ_decoupling(t):
    circuit = QuantumCircuit(5,2)
    
    circuit.barrier()
    circuit.x(2)
    circuit.x(3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.x(2)
    circuit.x(3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.delay(160,2)
    circuit.delay(160,3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.x(2)
    circuit.x(3)

    circuit.rz(np.pi,2)
    circuit.x(2)
    circuit.rz(np.pi,3)
    circuit.x(3)

    circuit.delay(192,2)
    circuit.delay(192,3)

    circuit.x(4)
    circuit.x(4)
    circuit.delay(432,4)
    circuit.x(4)
    circuit.x(4)
    circuit.delay(592,4)

    circuit.cx(0,1)
    circuit.barrier()

    return circuit

#dynamic decoupling 없는 CNOT circuit
def CNOT_circ(t):
    circuit = QuantumCircuit(5,2)
    
    circuit.barrier()
    circuit.cx(0,1)
    circuit.barrier()
    
    return circuit

# c = transpile(CNOT_circ(np.pi), backend = backend, optimization_level = 0, initial_layout = [1,3,0,2,5])
# c.draw()
# Circuit_to_schedule(c).draw()
import matplotlib
c = transpile(CNOT_circ_decoupling(np.pi), backend = backend, optimization_level = 0, initial_layout = [1,3,0,2,5])
# c.draw()
fig = Circuit_to_schedule(c).draw(plotter = 'mpl2d')
fig.savefig('Scheduler.png')

In [16]:
repeat = 5
shots = 8000
circuit_list = list()
circuit_list1 = list()
circuit_list2 = list()
t = np.pi

for j in range(4):
    a = QuantumCircuit(5, 2)
    b = QuantumCircuit(5, 2)
    if j == 0:
        state = '00'
    elif j == 1:
        a.x(1)
        b.x(1)
        state = '01'
    elif j == 2:
        a.x(0)
        b.x(0)
        state = '10'
    elif j == 3:
        a.x(0)
        a.x(1)
        b.x(0)
        b.x(1)
        state = '11'
    for k in range(3):
        a.compose(CNOT_circ(t), inplace = True)
        b.compose(CNOT_circ_decoupling(t), inplace = True)
    a.measure(0, 1)
    a.measure(1, 0)
    b.measure(0, 1)
    b.measure(1, 0)
    a_bind = transpile(a, backend = backend, optimization_level = 0, initial_layout = [1,3,0,2,5])
    b_bind = transpile(b, backend = backend, optimization_level = 0, initial_layout = [1,3,0,2,5])
    circuit_list1.append(a_bind)
    circuit_list2.append(b_bind)

circuit_list = circuit_list1 + circuit_list2
    
job = backend.run(circuit_list, shots = shots)
result = job.result()
counts = result.get_counts()
for k in range(len(counts)):
    if k == 0:
        print("Results w/o decoupling :")
    elif k == 4:
        print("Results w/ decoupling :")
    if k%4 == 0:
        print("00 becomes :", counts[k])
    elif k%4 == 1:
        print("01 becomes :", counts[k])
    elif k%4 == 2:
        print("10 becomes :", counts[k])
    elif k%4 == 3:
        print("11 becomes :", counts[k])

Results w/o decoupling :
00 becomes : {'00': 7302, '01': 533, '10': 61, '11': 104}
01 becomes : {'00': 137, '01': 7693, '10': 86, '11': 84}
10 becomes : {'00': 77, '01': 59, '10': 115, '11': 7749}
11 becomes : {'00': 54, '01': 92, '10': 7357, '11': 497}
Results w/ decoupling :
00 becomes : {'00': 7247, '01': 502, '10': 143, '11': 108}
01 becomes : {'00': 174, '01': 7548, '10': 113, '11': 165}
10 becomes : {'00': 108, '01': 129, '10': 122, '11': 7641}
11 becomes : {'00': 133, '01': 123, '10': 7246, '11': 498}
